# Ingesting real time data from the International Space Station (ISS)

### 1. Drop the database if it exists, create a new database, switch to it, and then create a table.

### Database Name
In the following cell you will enter your email address as the database name. However, you will need to replace all characters that are not underscores or alpha numberics with underscores.

### Example:
If your email address is <span style="color:yellow">lorrin.smith-bates@singlestore.com </span> you would use  <span style="color:red;">**lorrin_smith_bates_singlestore_com**</span>

In [ ]:
%%sql
DROP DATABASE IF EXISTS <<your modified email address here>>;
CREATE DATABASE <<your modified email address here>>;
USE <<your modified email address here>>;
CREATE TABLE iss_location(
    name varchar(10),
    id int,
    latitude float,
    longitude float,
    velocity float,
    visibility varchar(20),
    footprint float,
    timestamp bigint,
    daynum float,
    solar_lat float,
    solar_lon float,
    units varchar(20),
    url varchar(255)
);

### 2. Create a SingleStore pipeline to ingest ISS data from a Kafka topic.

In [ ]:
%%sql

CREATE OR REPLACE PIPELINE iss_pipeline AS
    LOAD DATA kafka '100.25.125.23/iss'
    INTO TABLE iss_location
    FORMAT JSON;

### 3. Test the pipeline.

In [ ]:
%%sql

TEST PIPELINE iss_pipeline;

### 4. Start the Pipeline

In [ ]:
%%sql

START PIPELINE iss_pipeline;

### 5. Get the count of records. Run this a few time to see the number of records ingested.

In [ ]:
%%sql

SELECT COUNT(*) FROM iss_location;

### 6. Get the latest location record. Click the link to see the location of the ISS in Google Maps.

In [ ]:
%%sql

SELECT timestamp, url 
    FROM iss_location
    ORDER BY timestamp desc
    LIMIT 1;

### 7. Stop the pipeline and delete the data from the iss_location table.

In [ ]:
%%sql

STOP PIPELINE iss_pipeline;
DELETE FROM iss_location;

### 8. Change the pipeline offsets and interval.

In [ ]:
%%sql

ALTER PIPELINE iss_pipeline
  SET BATCH_INTERVAL 30000
  SET OFFSETS latest ;

### 9. Start the Pipeline again.

In [ ]:
%%sql

START PIPELINE iss_pipeline;

### 10. Count the records, notice how the records are populated now after alterning the pipeline.

In [ ]:
%%sql

SELECT COUNT(*) from iss_location;

### 11. Stop the pipeline

In [ ]:
%%sql

STOP PIPELINE iss_pipeline;

# Query Optimization

### 1. Restore the 'employees' database that has been backed up into a public S3 bucket

For the database name we'll prepend employee_ to the modified email address again.

In [37]:
%%sql
RESTORE DATABASE employees AS employees_<<your modified email address here>>
  FROM S3 'train.memsql.com/employee'
  CONFIG'{"region":"us-east-1"}'
  CREDENTIALS'{}';

++
||
++
++

### 2. Switch to the Employees database

In [38]:
%%sql
USE employees_<<your modified email address here>>;

++
||
++
++

### 3. Run a query that joins 4 tables and orders by 4 columns in 3 tables

In [ ]:
%%sql
%%sql

SELECT e.first_name, e.last_name, d.dept_name, t.title, t.from_date, t.to_date
  FROM employees e
  INNER JOIN dept_emp de ON e.emp_no=de.emp_no
  INNER JOIN departments d ON de.dept_no=d.dept_no
  INNER JOIN titles t ON e.emp_no=t.emp_no
  ORDER BY e.first_name, e.last_name, d.dept_name, t.from_date
  LIMIT 10;

### 4. Examine the query execution profile using EXPLAIN

In [ ]:
%%sql
%%sql

EXPLAIN SELECT e.first_name, e.last_name, d.dept_name, t.title, t.from_date, t.to_date
  FROM employees e
  INNER JOIN dept_emp de ON e.emp_no=de.emp_no
  INNER JOIN departments d ON de.dept_no=d.dept_no
  INNER JOIN titles t ON e.emp_no=t.emp_no
  ORDER BY e.first_name, e.last_name, d.dept_name, t.from_date
  LIMIT 10;

### 5. Profile the query by using PROFILE.

In [ ]:
%%sql
%%sql
PROFILE SELECT e.first_name, e.last_name, d.dept_name, t.title, t.from_date, t.to_date
  FROM employees e
  INNER JOIN dept_emp de ON e.emp_no=de.emp_no
  INNER JOIN departments d ON de.dept_no=d.dept_no
  INNER JOIN titles t ON e.emp_no=t.emp_no
  ORDER BY e.first_name, e.last_name, d.dept_name, t.from_date
  LIMIT 10;

### 6. Run SHOW PROFILE to view the statistics on an actual run of the query

In [ ]:
%%sql
%%sql
SHOW PROFILE;

### 7.  Run Visual Profile to see this the profile in a GUI format

## Query/Schema Tuning Exercise

Now that we've visualized our query execution plan, let's address some of the issues we've uncovered. 

### 1. Create a Reference table for departments

In [52]:
%%sql
CREATE REFERENCE TABLE departments_ref(
  dept_no CHAR(4) not null,
  dept_name varchar(40) not null,
  primary key (dept_no),
  key(dept_name)
);

INSERT INTO departments_ref (SELECT * FROM departments);

++
||
++
++

### 2. Profile the old and the new

In [ ]:
%%sql
%%sql
%%sql
-- CONTROL. Here is the original query. We can use this as our control in our experiment.
SELECT e.first_name, e.last_name, d.dept_name, t.title, t.from_date, t.to_date
  FROM employees e
  INNER JOIN dept_emp de ON e.emp_no=de.emp_no
  INNER JOIN departments d ON de.dept_no=d.dept_no
  INNER JOIN titles t ON e.emp_no=t.emp_no
  ORDER BY e.first_name, e.last_name, d.dept_name, t.from_date
  LIMIT 10;



-- IMPROVED. Here is the slightly more improved query with the departments_ref table
SELECT e.first_name, e.last_name, d.dept_name, t.title, t.from_date, t.to_date
  FROM employees e
  INNER JOIN dept_emp de ON e.emp_no=de.emp_no
  INNER JOIN departments_ref d ON de.dept_no=d.dept_no
  INNER JOIN titles t ON e.emp_no=t.emp_no
  ORDER BY e.first_name, e.last_name, d.dept_name, t.from_date
  LIMIT 10;

-- PROFILE them both and observe the differences.

### 3. Create a titles table with sort and shard keys defined.

In [57]:
%%sql
CREATE  TABLE titles_sharded (
  emp_no INT NOT NULL,
  title VARCHAR(50) NOT NULL,
  from_date DATE NOT NULL,
  to_date DATE,
  SORT KEY (emp_no),
  SHARD KEY (emp_no)
);

INSERT INTO titles_sharded (SELECT * FROM titles);

++
||
++
++

### 4. Add shard and sort keys to the dept_emp table

In [ ]:
%%sql
CREATE TABLE dept_emp_sharded(
  emp_no int not null,
  dept_no char(4) not null,
  from_date date not null,
  to_date date not null,
  SORT KEY (dept_no),
  SHARD KEY(emp_no),
  KEY (dept_no)
);

INSERT INTO dept_emp_sharded (SELECT * FROM dept_emp);

In [ ]:
%%sql
SELECT e.first_name, e.last_name, d.dept_name, t.title, t.from_date, t.to_date
  FROM employees e
  INNER JOIN dept_emp de ON e.emp_no=de.emp_no
  INNER JOIN departments_ref d ON de.dept_no=d.dept_no
  INNER JOIN titles_sharded t ON e.emp_no=t.emp_no
  ORDER BY e.first_name, e.last_name, d.dept_name, t.from_date
  LIMIT 10;

### 5. Add shard and sort keys to the employees table

In [65]:
%%sql
CREATE TABLE employees_sharded (
    emp_no INT NOT NULL,
    birth_date DATE NOT NULL,
    first_name VARCHAR(14) NOT NULL,
    last_name VARCHAR(16) NOT NULL,
    hire_date DATE NOT NULL,
    SORT KEY (emp_no),
    SHARD KEY (emp_no)
);

INSERT INTO employees_sharded (SELECT * FROM employees);

++
||
++
++

In [ ]:
%%sql
SELECT e.first_name, e.last_name, d.dept_name, t.title, t.from_date, t.to_date
  FROM employees_sharded e
  INNER JOIN dept_emp de ON e.emp_no=de.emp_no
  INNER JOIN departments_ref d ON de.dept_no=d.dept_no
  INNER JOIN titles_sharded t ON e.emp_no=t.emp_no
  ORDER BY e.first_name, e.last_name, d.dept_name, t.from_date
  LIMIT 10;